In [ ]:
from pnet import pnet_loader, Pnet
from util import util, sankey_diag

import torch
import numpy as np
import os
import random
import seaborn as sns
import pandas as pd
from sklearn import metrics
import scipy
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA

%load_ext autoreload
%autoreload 2

In [ ]:
datapath='/mnt/disks/pancan/data'

## Load mutation data

In [ ]:
maf = pd.read_csv(datapath+'/m1000/M1000_CCF.maf', sep='\t')
survival_data = pd.read_csv(datapath+'/m1000/M1000_survival_data.txt', sep='\t').set_index('Tumor_Sample_Barcode')
clinical_mapping = pd.read_csv(datapath+'/m1000/TCGA_clinical_mapping_and_pathologic_M.txt', sep='\t').set_index('Tumor_Sample_Barcode')

maf = maf[maf['Tumor_Sample_Barcode'].isin(clinical_mapping.index)].copy()
maf['Variant_Classification'] = [util.MUTATIONS_DICT[m] for m in maf['Variant_Classification']]
maf = maf[maf['Variant_Classification'] != 'Silent'].copy()

maf_grouped = maf.groupby('Tumor_Sample_Barcode')['Hugo_Symbol'].apply(set).reset_index(name='mut_list').set_index('Tumor_Sample_Barcode')
mutations = pd.DataFrame(columns = maf['Hugo_Symbol'].unique(), index = maf_grouped.index)
mutations.fillna(0, inplace = True)

for i, p in maf_grouped.iterrows():
    for mut in p['mut_list']:
        mutations.loc[i][mut] = 1

mutations = mutations.join(clinical_mapping[['mapping_patient_id']], how='inner').set_index('mapping_patient_id')

## Load expression data

In [ ]:
skcm_exp = pd.read_csv(datapath+'/skcm_tcga_pan_can_atlas_2018/data_mrna_seq_v2_rsem_zscores_ref_all_samples.txt',
                       sep='\t').dropna().set_index('Hugo_Symbol').drop(['Entrez_Gene_Id'], axis=1).T
skcm_exp.index = ['-'.join(ind.split('-')[:-1]) for ind in skcm_exp.index]

In [ ]:
# non_constant_genes = util.select_non_constant_genes(skcm_exp)
# highly_variable_genes = util.select_highly_variable_genes(skcm_exp)['Hugo_Symbol'].values
# selected_genes = ['TP53', 'AR', 'PTEN', 'NOTCH1']
# genes = list(set(highly_variable_genes).intersection(non_constant_genes)) + selected_genes
# skcm_exp = skcm_exp[genes].copy()

## Load prediction target

In [ ]:
mat_TCGA = pd.read_csv(datapath+'/m1000/mat_TCGA.tsv', sep='\t').set_index('Tumor_Sample_Barcode')

mat_TCGA = mat_TCGA.join(clinical_mapping[['mapping_patient_id']], how='inner').set_index('mapping_patient_id')

heterogeneity_y = pd.DataFrame(index=mat_TCGA.index, columns=['dichtomized_heterogeneity'],
                               data=[int(p > mat_TCGA['heterogeneity'].median()) 
                                     for p in mat_TCGA['heterogeneity'].values])

In [ ]:
plt.hist(mat_TCGA['heterogeneity'], bins=80)
plt.vlines([mat_TCGA['heterogeneity'].median()], color='r', ymin=0, ymax=mat_TCGA['heterogeneity'].count()/10)
plt.show()

In [ ]:
# heterogeneity_y_ex = pd.qcut(mat_TCGA['heterogeneity'], 3, labels=['low', 'mid', 'high'])
# heterogeneity_y_ex = heterogeneity_y_ex[heterogeneity_y_ex!='mid']
# heterogeneity_y_ex = pd.DataFrame(index=heterogeneity_y_ex.index, columns=['dichtomized_heterogeneity'],
#                                data=[int(p == 'high') 
#                                      for p in heterogeneity_y_ex])

## Validation with Liu cohort

In [ ]:
sample_mapping = pd.read_csv(datapath+'/mel_dfci_2019/data_clinical_sample.txt', sep='\t').set_index('#Patient Identifier')
liu_heterogeneity = pd.DataFrame(sample_mapping['Heterogeneity']).iloc[4:].astype(float)
liu_heterogeneity_y = pd.DataFrame(index=liu_heterogeneity.index, columns=['dichtomized_ploidy'],
                               data=[int(p > liu_heterogeneity['Heterogeneity'].median()) 
                                     for p in liu_heterogeneity['Heterogeneity'].values])

In [ ]:
liu_maf = pd.read_csv(datapath+'/mel_dfci_2019/all_muts_12_1_2020_ref_alt_counts_added.maf', sep='\t')
liu_clinical_mapping = pd.read_csv(datapath+'/mel_dfci_2019/data_clinical_sample.txt',
                                   sep='\t').set_index('#Patient Identifier')

liu_maf = liu_maf[liu_maf['Patient'].isin(liu_clinical_mapping.index)].copy()
liu_maf['Variant_Classification'] = [util.MUTATIONS_DICT[m] for m in liu_maf['Variant_Classification']]
liu_maf = liu_maf[liu_maf['Variant_Classification'] != 'Silent'].copy()

liu_maf_grouped = liu_maf.groupby('Patient')['Hugo_Symbol'].apply(set).reset_index(name='mut_list').set_index('Patient')
liu_mutations = pd.DataFrame(columns = liu_maf['Hugo_Symbol'].unique(), index = liu_maf_grouped.index)
liu_mutations.fillna(0, inplace = True)

for i, p in liu_maf_grouped.iterrows():
    for mut in p['mut_list']:
        liu_mutations.loc[i][mut] = 1

In [ ]:
liu_rna = pd.read_csv(datapath+'/mel_dfci_2019/data_RNA_Seq_expression_tpm_all_sample_Zscores.txt',
                          delimiter='\t').set_index('Hugo_Symbol').T.drop('Entrez_Gene_Id').dropna(axis=1)
liu_rna = sample_mapping[['Sample Identifier']].join(liu_rna, on='Sample Identifier',
                                                     how='inner').drop('Sample Identifier', axis=1)
canc_genes = list(pd.read_csv('../../pnet_database/genes/cancer_genes.txt').values.reshape(-1))
genes = list(set.intersection(set(skcm_exp.columns), set(mutations.columns), set(liu_rna.columns), set(liu_mutations.columns),
                              set(canc_genes)))
skcm_exp = skcm_exp[genes].copy()
liu_rna = liu_rna[genes].copy()

## Generate folds

In [ ]:
genetic_data = {'rna': skcm_exp, 'mut': mutations}

Run this only if folds do not exist yet:

In [ ]:
# melanoma_inds = pnet_loader.get_indicies(genetic_data, heterogeneity_y)
# random.shuffle(melanoma_inds)

# def chunks(lst, n):
#     """Yield successive n-sized chunks from lst."""
#     for i in range(0, len(lst), n):
#         yield lst[i:i + n]
        
# test_splits = chunks(melanoma_inds, int(len(melanoma_inds)/10)+1)
# for i, s in enumerate(test_splits):
#     train_dataset, test_dataset = pnet_loader.generate_train_test(genetic_data, heterogeneity_y, test_inds=s)
#     train_dataset.save_indicies(datapath+'/splits/skcm_heterogeneity/train_set_{}.csv'.format(i))
#     test_dataset.save_indicies(datapath+'/splits/skcm_heterogeneity/test_set_{}.csv'.format(i))

## Train with run()

In [ ]:
genetic_data = {'rna': skcm_exp, 'mut': mutations}
val_genetic_data = {'rna': liu_rna, 'mut': liu_mutations}

In [ ]:
val_inds = pnet_loader.get_indicies(val_genetic_data, liu_heterogeneity_y)
val_dataset = pnet_loader.PnetDataset(val_genetic_data, liu_heterogeneity_y, val_inds)

In [ ]:
for i in range(10):
    train_inds = list(pd.read_csv(datapath+'/splits/skcm_heterogeneity/train_set_{}.csv'.format(i))['indicies'])
    test_inds = list(pd.read_csv(datapath+'/splits/skcm_heterogeneity/test_set_{}.csv'.format(i))['indicies'])
    model, train_scores, test_scores, train_dataset, test_dataset = Pnet.run(genetic_data, heterogeneity_y, seed=0,
                                                                             dropout=0.3, lr=1e-4, weight_decay=1,
                                                                             batch_size=64, epochs=300, early_stopping=True,
                                                                             train_inds=train_inds, test_inds=test_inds)
    plt.clf()
    Pnet.evaluate_interpret_save(model, test_dataset, '/mnt/disks/pancan/pnet/results/heterogeneity/tcga_skcm/run{}'.format(i))
    plt.clf()
    Pnet.evaluate_interpret_save(model, val_dataset, '/mnt/disks/pancan/pnet/results/heterogeneity/liu_val_skcm/run{}'.format(i))

In [ ]:
tcga_aucs = []
for i in range(10):
    auc = torch.load('/mnt/disks/pancan/pnet/results/heterogeneity/tcga_skcm/run{}/AUC.pt'.format(i))
    tcga_aucs.append(auc.item())

In [ ]:
liu_aucs = []
for i in range(10):
    auc = torch.load('/mnt/disks/pancan/pnet/results/heterogeneity/liu_val_skcm/run{}/AUC.pt'.format(i))
    liu_aucs.append(auc.item())

In [ ]:
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False

In [ ]:
plt.clf()
auc_df = pd.DataFrame({'TCGA \n 10 fold CV': tcga_aucs, 'LIU \n external validation': liu_aucs})
auc_df.boxplot(showfliers=False,
           color=dict(boxes='grey', whiskers='silver', medians='dimgray', caps='silver'), widths=0.6, patch_artist=True)
plt.ylim((0,1))
plt.grid(False)
plt.legend(['median AUC'], loc='lower right')
plt.ylabel('AUC')
plt.xlabel('Cohort')
plt.axhline(y=0.5, color='silver', linestyle='-')
plt.savefig('../figures/SKCM_heterogeneity_TCGA_vs_Liu.pdf')
plt.show()

In [ ]:
for i in range(10):
    skcm_path = '/mnt/disks/pancan/pnet/results/heterogeneity/tcga_skcm_rf/run{}'.format(i)
    val_path = '/mnt/disks/pancan/pnet/results/heterogeneity/liu_val_skcm_rf/run{}'.format(i)
    if not os.path.exists(skcm_path):
        os.makedirs(skcm_path)
    if not os.path.exists(val_path):
        os.makedirs(val_path)
    train_inds = list(pd.read_csv(datapath+'/splits/skcm_heterogeneity/train_set_{}.csv'.format(i))['indicies'])
    test_inds = list(pd.read_csv(datapath+'/splits/skcm_heterogeneity/test_set_{}.csv'.format(i))['indicies'])
    train_dataset, test_dataset = pnet_loader.generate_train_test(genetic_data, target=heterogeneity_y, train_inds=train_inds, test_inds=test_inds)
    
    x_train = train_dataset.x
    additional_train = train_dataset.additional
    y_train = train_dataset.y.ravel()
    x_test = test_dataset.x
    additional_test = test_dataset.additional
    y_test = test_dataset.y.ravel()
    x_val = val_dataset.x
    additional_val = val_dataset.additional
    y_val = val_dataset.y.ravel()
    
    rfc = RandomForestClassifier(max_depth=None, random_state=0)
    rfc.fit(x_train, y_train)
    preds = rfc.predict(x_test)
    preds_prob = rfc.predict_proba(x_test)
    plt.clf()
    auc = util.get_auc(torch.tensor(preds_prob[:,1], dtype=torch.float), y_test, save=skcm_path+'/auc_curve.pdf')
    importances = rfc.feature_importances_
    forest_importances = pd.Series(importances, index=test_dataset.input_df.columns)
    forest_importances.to_csv(skcm_path+'/gene_feature_importances.csv')
    torch.save(auc, skcm_path+'/AUC.pt')
    
    preds = rfc.predict(x_val)
    preds_prob = rfc.predict_proba(x_val)
    plt.clf()
    auc = util.get_auc(torch.tensor(preds_prob[:,1], dtype=torch.float), y_val, save=val_path+'/auc_curve.pdf')
    importances = rfc.feature_importances_
    forest_importances = pd.Series(importances, index=test_dataset.input_df.columns)
    forest_importances.to_csv(val_path+'/gene_feature_importances.csv')
    torch.save(auc, val_path+'/AUC.pt')

In [ ]:
tcga_rf_aucs = []
for i in range(10):
    auc = torch.load('/mnt/disks/pancan/pnet/results/heterogeneity/tcga_skcm_rf/run{}/AUC.pt'.format(i))
    tcga_rf_aucs.append(auc.item())

liu_rf_aucs = []
for i in range(10):
    auc = torch.load('/mnt/disks/pancan/pnet/results/heterogeneity/liu_val_skcm_rf/run{}/AUC.pt'.format(i))
    liu_rf_aucs.append(auc.item())
    
plt.clf()
auc_df = pd.DataFrame({'SKCM TCGA \n 10 fold CV': tcga_aucs, 'Liu 2019 \n external validation': liu_aucs,
                       #'RF TCGA \n 10 fold CV': tcga_rf_aucs, 'RF LIU \n external validation': liu_rf_aucs
                      })
auc_df.boxplot(showfliers=False,
           color=dict(boxes='grey', whiskers='silver', medians='dimgray', caps='silver'), widths=0.6, patch_artist=True, showmeans=True, 
               meanprops={"marker":".","markerfacecolor":"gainsboro", "markeredgecolor":"gainsboro"})
plt.ylim((0,1))
plt.axhline(y=0.5, color='silver', linestyle='-')
plt.grid(False)
plt.ylabel('AUC')
plt.xlabel('Cohort')
plt.savefig('../figures/SKCM_heterogeneity_TCGA_vs_Liu_rf.pdf', bbox_inches='tight')
plt.show()

In [ ]:
all_imps = pd.DataFrame(columns=['SNR', 'Model', 'Layer', 'imp_mean', 'imp_std'])
for m in ['Pnet']:
    for l in ['gene_feature', 'gene', 'layer_0', 'layer_1', 'layer_2', 'layer_3']:
        df_imps = pd.DataFrame()
        df_ranks = pd.DataFrame()
        for i in range(10):
            imps = pd.read_csv('/mnt/disks/pancan/pnet/results/heterogeneity/tcga_skcm/run{}/{}_importances.csv'.format(i, l)).set_index('Unnamed: 0')
            imps = abs(imps.join(heterogeneity_y).groupby('dichtomized_heterogeneity').mean().diff(axis=0).iloc[1])
            ranks = imps.rank(ascending=False)
            df_imps['run{}'.format(i)] = imps
            df_ranks['run{}'.format(i)] = ranks
            
        imp_mean = df_imps.mean(axis=1)
        imp_std = df_imps.std(axis=1)
        snr = imp_mean/(imp_std+1e-9)
        melted_imps = snr.to_frame('SNR')
        melted_imps['imp_mean'] = imp_mean
        melted_imps['imp_std'] = imp_std
        melted_imps['Model'] = m
        melted_imps['Layer'] = l
        all_imps = pd.concat([all_imps, melted_imps])

        
layer_numeric = {'gene_feature':-1, 'gene':0, 'layer_0':1, 'layer_1':2, 'layer_2':3, 'layer_3':4}
all_imps['Numeric Layer'] = all_imps['Layer'].apply(lambda x: layer_numeric[x])
all_imps['Z'] = all_imps.groupby('Layer')['SNR'].transform(lambda x: (x - x.mean()) / x.std())
all_imps['p_val'] = scipy.stats.norm.sf(abs(all_imps['Z']))

all_pathway_imps = all_imps[~all_imps['Layer'].isin(['gene_feature', 'gene'])]
all_imps[all_imps['Z'] > 3]

In [ ]:
rf_df_imp = pd.DataFrame()
rf_df_rnk = pd.DataFrame()
for i in range(10):
    gene_imps = pd.read_csv('/mnt/disks/pancan/pnet/results/heterogeneity/liu_val_skcm_rf/run{}/gene_feature_importances.csv'.format(i))
    gene_imps['Unnamed: 0'] = gene_imps['Unnamed: 0'].apply(lambda x: x.split('_')[0])
    gene_imps = gene_imps.set_index('Unnamed: 0')
    gene_imps = pd.DataFrame(gene_imps.values.reshape((2, int(gene_imps.shape[0]/2))).T.sum(axis=1),
                             index=gene_imps.index[:int(gene_imps.shape[0]/2)],
                             columns=['gene_imp'])
    rf_df_imp['run_{}'.format(i)] = gene_imps
    rf_df_rnk['run_{}'.format(i)] = gene_imps.rank(ascending=False)
rf_rank_var = rf_df_rnk.loc[rf_df_rnk.mean(axis=1).nsmallest(50).index].std(axis=1).nsmallest(100).median()
rf_imps = rf_df_imp.mean(axis=1)

In [ ]:
sns.scatterplot(x=all_imps[all_imps['Layer'].isin(['gene'])]['SNR'], y=rf_imps)

In [ ]:
import gseapy as gp
ss = gp.ssgsea(data=pd.DataFrame(rf_imps), 
               gene_sets='/mnt/disks/pancan/pnet/data/reactome/ReactomePathways.gmt', 
               outdir='/mnt/disks/pancan/pnet/results/heterogeneity/liu_val_skcm/ssgsea')
pathway_scores = ss.res2d.pivot(index='Term', columns='Name', values='NES').T

In [ ]:
pathway_scores.T['0'].astype(float).nlargest(20)